### Import libraries & utils

In [ ]:
using Gen

In [ ]:
using WAV
include("../tools/plotting.jl")
include("../model/gammatonegram.jl");
include("../model/time_helpers.jl");
include("../model/extra_distributions.jl");

### Get some default model parameters:

In [ ]:
source_params, steps, gtg_params, obs_noise = include("../params/base.jl")
sr = 2000.0
gtg_params["dB_threshold"] = 0.0
wts, = gtg_weights(sr, gtg_params);

## Model

### Embed a sound within a larger scene

In [ ]:
function embed_in_scene(scene_length, sr, wave, onset)
  n_samples = Int(floor(sr * scene_length))
  scene_wave = zeros(n_samples)
  sample_start = max(1, Int(floor(onset * sr)))
  sample_finish = min(sample_start + length(wave), length(scene_wave))
  scene_wave[sample_start:sample_finish-1] = wave[1:length(sample_start:sample_finish-1)]
  return scene_wave
end

### Generate white noise sound

In [ ]:
@gen function generate_single_noise(scene_length, steps, sr)
  onset ~ uniform(0, scene_length)
  duration ~ uniform(0.1, 1.0)
  amp ~ normal(10.0, 8.0)
  times, t, f = get_gp_spectrotemporal([onset, onset+duration], steps, sr)
  noise_wave = generate_noise(transpose(reshape(fill(amp, length(times)), (length(f), length(t)))), duration, steps, sr, 1e-6)
  return embed_in_scene(scene_length, sr, noise_wave, onset)
end

### Generate tone with pitch

In [ ]:
@gen function generate_single_tone(scene_length, step_size, sr)
  step_size = step_size["t"]
  erb ~ uniform(0.4, 37.0)
  onset ~ uniform(0.0, scene_length)
  duration ~ uniform(0.1, 1.0)
  times = get_element_gp_times([onset, onset + duration], step_size)
  wave = generate_tone(fill(erb, length(times)), fill(50.0, length(times)), duration, step_size, sr, 1.0e-6)
  return embed_in_scene(scene_length, sr, wave, onset)
end

### Generate sound (tone or noise)

In [ ]:
@gen function generate_single_sound(scene_length, steps, sr)
    is_noise ~ bernoulli(0.4)
    if is_noise
        wave = {*} ~ generate_single_noise(scene_length, steps, sr)
    else
        wave = {*} ~ generate_single_tone(scene_length, steps, sr)
    end
    return wave
end
generate_sounds = Map(generate_single_sound);

### Generate scene with multiple sounds

In [ ]:
@gen (static) function generate_scene(scene_duration, wts, audio_sr, steps, gtg_params)
  n_tones ~ uniform_discrete(1, 4)
  
  waves ~ generate_sounds(fill(scene_duration, n_tones), fill(steps, n_tones), fill(audio_sr, n_tones))
  n_samples = Int(floor(scene_duration * audio_sr))
  scene_wave = reduce(+, waves; init=zeros(n_samples))
  scene_gram, = gammatonegram(scene_wave, wts, audio_sr, gtg_params)
  scene ~ noisy_matrix(scene_gram, 1.0)
  return scene_gram, scene_wave, waves
end

In [ ]:
@load_generated_functions()

## Visualization and playback

In [ ]:
function vis_and_write_wave(tr, title)
  duration, _, sr, = get_args(tr)
  gram, scene_wave, = get_retval(tr)
  wavwrite(scene_wave/maximum(abs.(scene_wave)), title, Fs=sr)
  plot_gtg(gram, duration, sr, 0, 100)
end

Some default arguments to `generate_scene`:

In [ ]:
args = (2.0, wts, sr, steps, gtg_params);

Generate & visualize a trace:

In [ ]:
tr = simulate(generate_scene, args);
vis_and_write_wave(tr, "simulated_scene.wav")

In [ ]:
; afplay simulated_scene.wav

### Generate Auditory Illusion

In [ ]:
function tones_with_noise(amp)
    cm = choicemap(:n_tones => 3,
              (:waves => 1 => :is_noise) => false,
              (:waves => 1 => :erb) => 10.0,
              (:waves => 1 => :onset) => 0.5,
              (:waves => 1 => :duration) => 0.3,
              (:waves => 2 => :is_noise) => false,
              (:waves => 2 => :erb) => 10.0,
              (:waves => 2 => :onset) => 1.1,
              (:waves => 2 => :duration) => 0.3,
              (:waves => 3 => :is_noise) => true,
              (:waves => 3 => :amp) => amp,
              (:waves => 3 => :onset) => 0.8,
              (:waves => 3 => :duration) => 0.3)
    tr, = generate(generate_scene, args, cm)
    return tr
end

In [ ]:
trr = tones_with_noise(10.0);
get_score(trr);

In [ ]:
vis_and_write_wave(trr, "trr.wav")

In [ ]:
; afplay trr.wav

## Generic Inference

In [ ]:
function do_generic_inference(tr, iters)
  for i=1:iters
    for j=1:tr[:n_tones]
        tr, = mh(tr, select(:waves => j))
        if tr[:waves => j => :is_noise]
          tr, = mh(tr, select(:waves => j => :amp))
        else
          tr, = mh(tr, select(:waves => j => :erb))
        end
        if bernoulli(0.5)
          tr, = mh(tr, select(:waves => j => :onset, :waves => j => :duration))
        else
          tr, = mh(tr, select(:waves => j => :onset))
          tr, = mh(tr, select(:waves => j => :duration))
        end
      end
    tr, = mh(tr, select(:n_tones))
  end
  tr
end

### Run generic inference on audio illusion

Generate a random trace where the observations are the same as the ground truth:

In [ ]:
observations = choicemap(:scene => trr[:scene])
initial_tr, = generate(generate_scene, args, observations);

Run 100 inference iterations:

In [ ]:
inferred_tr = do_generic_inference(initial_tr, 200);
get_score(inferred_tr)

In [ ]:
vis_and_write_wave(inferred_tr, "inferred.wav")

### Birth/death Inference

Proposal distribution:

In [ ]:
@gen function birth_death_proposal(tr)
    do_birth ~ bernoulli(0.5)
    if do_birth
        idx ~ uniform_discrete(1, tr[:kernel => :n_tones] + 1)
    else
        idx ~ uniform_discrete(1, tr[:kernel => :n_tones])
    end
end

In [ ]:
@transform (x, y) to (x_new, y_reverse) begin
  do_birth = @read(fwd_prop_tr[:do_birth], :disc)
  idx = @read(fwd_prop_tr[:idx], :disc)
  num = @read(old_tr[:kernel => :n_tones], :disc)
  if do_birth
    # increase the number of tones:
    @write(x_new[:n_tones], old_num + 1, :disc)
    
    # copy parameters for old tones up to a new index
    # problem: THIS IS SLOW!
    for i=new_idx:old_num
      @copy(x[:waves => i], x[:waves => i+1])
    end
    
    # generate a new tone at the new index
    @regenerate(:waves => new_idx)
  else
    # decrease the number of tones:
    @write(x_new[:n_tones], old_num - 1, :disc)
    
    # copy parameters for old tones up to a new index
    for i=(death_idx+1):old_num
      @copy(x[:waves => i], x[:waves => i-1])
    end
    
    # "save the old sound to the reverse proposal"
    @save_for_reverse_regenerate(:waves => new_idx)
  end
  
  # reverse move:
  @write(bwd_prop_tr[:do_birth], !do_birth, :disc)
  @write(bwd_prop_tr[:idx], idx, :disc)
end